# A. Librerias y rutas

In [3]:
import datetime
import pandas as pd
import os
import time
import json
import wget
import zipfile
import requests
import platform

import datetime
import pandas as pd
from browsermobproxy import Server

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [5]:
# 1. Definir rutas y sistema operativo
os_platform = platform.system().lower()

# Definir la carpeta de descargas para los archivos JSON
LOG_FILE_PATH = os.path.join("./scraping_log.txt")

# 2. Configurar rutas del proyecto según el sistema operativo
if os_platform == "darwin":
    BASE_PATH = "/Volumes/Projects/GitHubProjects/UNI-SCRAPING"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")
elif os_platform == "linux":
    BASE_PATH = "/home/brew_test_gcp_01/Desktop/WallyScraper"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")
elif os_platform == "windows":
    BASE_PATH = "D:\\GithubProjects\\Tesis\\Transparencia"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")

if os_platform == "darwin":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "darwin", "chromedriver")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy")
elif os_platform == "linux":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "linux", "chromedriver")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy")
elif os_platform == "windows":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "windows", "chromedriver.exe")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy.bat")


# 3. Imprimir variables
print("ESTAMOS EN ---> ", os_platform)
print("CHROME_DRIVER_PATH: ", CHROMEDRIVER_PATH)
print("BROWSERMOB_PATH: ", BROWSERMOB_PATH)
print("")

ESTAMOS EN --->  darwin
CHROME_DRIVER_PATH:  /Volumes/Projects/GitHubProjects/UNI-SCRAPING/chromedriver/darwin/chromedriver
BROWSERMOB_PATH:  /Volumes/Projects/GitHubProjects/UNI-SCRAPING/browsermob-proxy-2.1.4/bin/browsermob-proxy



# B. Funciones